Author: Alejandro C Parra Garcia

In [1]:
#Import the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import tarfile
from bs4 import BeautifulSoup
import re
import string

Lets import the emails

In [2]:
#Change the path to the Location of the files
path="D:\\Personal\\Alex\\Uni\\MS IA\\1 Curso\\FALL\\Natural Language Processing\\Module 1\\Assignments\\Corpus"

In [3]:
tarfile_spam=tarfile.open(path+"\\20030228_spam.tar.bz2", mode='r:bz2')
tarfile_spam_2=tarfile.open(path+"\\20030228_spam_2.tar.bz2", mode='r:bz2')
tarfile_easy_ham=tarfile.open(path+"\\20030228_easy_ham.tar.bz2", mode='r:bz2')
tarfile_easy_ham_2=tarfile.open(path+"\\20030228_easy_ham_2.tar.bz2", mode='r:bz2')
tarfile_hard_ham=tarfile.open(path+"\\20030228_hard_ham.tar.bz2", mode='r:bz2')

Let's create the dataframe

In [4]:
df = pd.DataFrame(columns=['mail','spam'])
df

,mail,spam


In [5]:
def add_email_to_df(tarfile, spam):
    global df
    for elem in tarfile:
        if (elem.isreg()): #Check if is a file
            if(elem.name!="cmds"):#Ignore the file cmds
                f = tarfile.extractfile(elem)
                try:
                    mail=f.read().decode("utf-8")
                    df = df.append({'mail': mail, 'spam':spam}, ignore_index=True)
                except UnicodeDecodeError:
                    continue

In [6]:
add_email_to_df(tarfile_spam,1) #Spam
add_email_to_df(tarfile_spam_2,1) #Spam
add_email_to_df(tarfile_easy_ham,0) #Not Spam
add_email_to_df(tarfile_easy_ham_2,0) #Not Spam
add_email_to_df(tarfile_hard_ham,0) #Not Spam
df

,mail,spam
0,From 12a1mailbot1@web.de Thu Aug 22 13:17:22 ...,1
1,From ilug-admin@linux.ie Thu Aug 22 13:27:39 ...,1
2,From sabrina@mx3.1premio.com Thu Aug 22 14:44...,1
3,From wsup@playful.com Thu Aug 22 16:17:00 200...,1
4,From social-admin@linux.ie Thu Aug 22 16:37:3...,1
...,...,...
5547,From tesco.ie@ctejmi.cjsias.bwmmxq.bounce.12hs...,0
5548,From senews-nov@euromailer.lists.apple.com Th...,0
5549,From bounce-neatnettricks-2424157@silver.lyris...,0
5550,From sporadic@fuckedcompany.com Wed Oct 30 21...,0


In [7]:
#Check for empy values. And delete them
if(df.isnull().values.any()==False):
    print("No empty values")
else:
    print("Empty values")
    df=df.dropna()

df.shape

No empty values


(5552, 2)

In [8]:
#Download the spotwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
#Clean Up
    #Convert HTML to txt
    #Remove punctuation
    #Lower Case
def clean_up(mail):
    if(mail.find('Content-Type: text/html')!=-1):
        #Is HTML
        #Convert HTML to Text
        soup = BeautifulSoup(mail)
        mail= soup.get_text() 
        #mail= re.sub(r'\<.*?\>','',mail)#Delete the <...>
    
    #Remove Punctuation
    mail= re.sub(r'\W',' ',mail)
    
    #Lower Case
    mail=mail.lower()
    
    return (mail)

In [10]:
#Tokenice the text
def tokenizations(mail):
    
    ret=nltk.word_tokenize(mail)
            
    return (ret)

In [11]:
#remove stopwords
def remove_stopwords(mail):
    stopwords_var=set(stopwords.words('english'))
    ret=[]
    for elem in mail:
        if (elem not in stopwords_var):
            ret.append(elem)
    return (ret)

In [12]:
#Stemmer
def stemming(mail):
    stemmer_var=SnowballStemmer('english')
    ret=[]
    for elem in mail:
        ret.append(stemmer_var.stem(elem))
    return (ret)

In [13]:
#data treatment
df['mail']= df.apply(lambda elem: clean_up(elem['mail']),axis=1)
df['mail']= df.apply(lambda elem: tokenizations(elem['mail']),axis=1)
df['mail']= df.apply(lambda elem: remove_stopwords(elem['mail']),axis=1)
df['mail']= df.apply(lambda elem: stemming(elem['mail']),axis=1)

#elem=df.iloc[0]['mail']
#print(elem)

In [14]:

list_mail=[]
for i in df['mail']:
    list_mail.append(" ".join(i))


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [16]:
#Create the bag of words
bag_of_words=CountVectorizer().fit_transform(list_mail)

#df_new = pd.DataFrame(bag_of_words)

In [22]:
#Split the data. 70% for training 30% for test

x_train, x_test, y_train, y_test = train_test_split(bag_of_words, df['spam'].astype('int'), test_size=0.30, random_state=0)

In [23]:
bag_of_words.shape

(5552, 135386)

In [25]:
#Use Multinomial Naive Bayes
mnb = MultinomialNB()
mnb.fit(x_train,y_train)


MultinomialNB()

In [29]:
prediction=mnb.predict(x_train)

print(metrics.classification_report(y_train,prediction))
print("Confusion matrix:")
print(metrics.confusion_matrix(y_train,prediction))
print("Accuracy:")
print(metrics.accuracy_score(y_train,prediction))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2697
           1       0.99      0.94      0.96      1189

    accuracy                           0.98      3886
   macro avg       0.98      0.97      0.97      3886
weighted avg       0.98      0.98      0.98      3886

Confusion matrix:
[[2689    8]
 [  74 1115]]
Accuracy:
0.9788986103962944


In [31]:
#Lets test the test data

prediction_test=mnb.predict(x_test)

print(metrics.classification_report(y_test, prediction_test))
print("Confusion matrix:")
print(metrics.confusion_matrix(y_test, prediction_test))
print("Accuracy:")
print(metrics.accuracy_score(y_test, prediction_test))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1161
           1       0.97      0.92      0.95       505

    accuracy                           0.97      1666
   macro avg       0.97      0.96      0.96      1666
weighted avg       0.97      0.97      0.97      1666

Confusion matrix:
[[1148   13]
 [  38  467]]
Accuracy:
0.9693877551020408
